# Spark Machine Learning

En este laboratorio veremos como crear, entrenar y consultar un modelo predictivo con Spark

Los datos siguientes muestran la temperatura objetivo y la temperatura real de algunos edificios con sistemas HVAC instalados. La columna System representa el identificador del sistema y la columna SystemAge, el número de años que lleva el sistema HVAC instalado en el edificio. Los datos de este tutorial se usan para predecir si un edificio será más cálido o frío en función de la temperatura objetivo, dados un identificador del sistema y la antigüedad del sistema

In [ ]:
# Cargamos los objetos y funciones necesarias

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.sql import Row

import os
import sys

from pyspark.sql.types import *

from pyspark.mllib.classification import LogisticRegressionWithSGD
from pyspark.mllib.regression import LabeledPoint
from numpy import array



Cargamos los datos del .csv. En el fragmento de código, se define una función que compara la temperatura real con la temperatura de destino. Si la temperatura real es mayor, el edificio está **cálido**, lo que viene indicado por el valor 1.0. De lo contrario, el edificio está **frío**, lo que se indica con el valor 0.0.

In [ ]:
# Definimos un tipo llamado LabelDocument

LabeledDocument = Row("BuildingID", "SystemInfo", "label")

# Definimos una función que parsea el CSV y devuelve un objeto de tipo LabelDocument

def parseDocument(line):
    values = [str(x) for x in line.split(',')]
    if (values[3] > values[2]):
        hot = 1.0
    else:
        hot = 0.0        

    textValue = str(values[4]) + " " + str(values[5])

    return LabeledDocument((values[6]), textValue, hot)

# Cargamos el fichero HVAC.csv, y lo parseamos utilizando la función definida anteriormente

data = sc.textFile("wasb:///HdiSamples/HdiSamples/SensorSampleData/hvac/HVAC.csv")

documents = data.filter(lambda s: "Date" not in s).map(parseDocument)
training = documents.toDF()

Un **pipeline** es es un flujo de trabajo completo que combina varios algoritmos de ML. Los pipelines definen las fases y el orden de un proceso de ML. 

Un **transformador** es un algoritmo que transforma un elemento DataFrame en otro. Por ejemplo, un transformador de características puede leer una columna de un elemento DataFrame, asignarla a otra columna y generar un nuevo elemento DataFrame con la columna asignada anexada

Un **estimador** es una abstracción de algoritmos de aprendizaje y es responsable del ajuste o el aprendizaje en un conjunto de datos para producir un transformado

En nuestro caso generaremos un pipeline que consta de tres fases: tokenizer, hashingTF (ambos son transformadores) y lr:

1. En la primera fase, Tokenizer, se divide la columna de entrada SystemInfo (que consta de los valores de identificador y antigüedad del sistema) en una columna de salida words. Esta nueva columna words se agrega a DataFrame

2. En la segunda fase, HashingTF, se convierte la nueva columna words en vectores de característica. Esta nueva columna features se agrega al elemento DataFrame. Estas dos primeras fases son transformadores. 

3. La tercera fase, LogisticRegression, es un estimador, por lo que la canalización llama al método LogisticRegression.fit() para generar un modelo LogisticRegressionModel.



In [ ]:
tokenizer = Tokenizer(inputCol="SystemInfo", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.01)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

Creamos un modelo entrenando el pipeline

In [ ]:
model = pipeline.fit(training)

Comprobamos los datos

In [ ]:
training.show()

Preparamos un conjunto de datos con el que ejecutar el modelo entrenado. Para ello, deberemos de pasar un identificador del sistema y la antigüedad del sistema (representados como SystemInfo en la salida de entrenamiento), y el modelo predecirá si el edificio con ese identificador y esa antigüedad del sistema es más cálido (indicado por 1.0) o frío (indicado por 0.0).

In [ ]:
# SystemInfo es una combinación de system ID y systemage 
Document = Row("id", "SystemInfo")
test = sc.parallelize([(1L, "20 25"),
                (2L, "4 15"),
                (3L, "16 9"),
                (4L, "9 22"),
                (5L, "17 10"),
                (6L, "7 22")]) \
    .map(lambda x: Document(*x)).toDF()

Por último, realizamos predicciones basadas en los datos de prueba

In [ ]:
# Hacemos predicciones con los datos de prueba y mostramos los resultados
prediction = model.transform(test)
selected = prediction.select("SystemInfo", "prediction", "probability")
for row in selected.collect():
    print row

En la primera fila de la predicción, podemos ver que, en un sistema HVAC con el identificador 20 y una antigüedad de 25 años, el edificio tiene una temperatura cálida (prediction=1.0). El primer valor de DenseVector (0.49999) corresponde a la predicción 0.0 y el segundo, (0.5001), corresponde a la predicción 1.0. En la salida, aunque el segundo valor solo es levemente superior, el modelo muestra prediction=1.0.